In [1]:
import torch
from torch import nn
from torchtext.data import Field, BucketIterator
from torchtext.datasets import CoNLL2000Chunking

from tqdm import tqdm

In [2]:
TEXT = Field(lower=True, batch_first=True)
LABEL = Field(batch_first=True)

In [3]:
train,val,test = CoNLL2000Chunking.splits(fields=[('text',TEXT),('label',LABEL)])

In [4]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [5]:
trainloader, testloader = BucketIterator.splits((train,test),batch_size=32)

In [17]:
class BiLSTM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim

        self.word_embeds = nn.Embedding(len(TEXT.vocab), embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,
                            num_layers=2, bidirectional=True)
        self.ln = nn.LayerNorm(hidden_dim*2)
        self.hidden2tag = nn.Linear(hidden_dim*2, len(LABEL.vocab))

    def forward(self, sentence):
        tmp = self.word_embeds(sentence)
        out, (ht,ct) = self.lstm(tmp)
        out = self.ln(out)
        out = self.hidden2tag(out)
        
        return out

In [20]:
model = BiLSTM(128,128).cuda()

In [21]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

In [19]:
# Check predictions before training
with torch.no_grad():
    model.eval()
    for i in trainloader:
        print(model(i.text.cuda()).shape)
        break

torch.Size([32, 46, 46])


In [22]:
model.train()
for epoch in range(10): 
    running_loss = 0.0
    for ss in tqdm(trainloader):
        sentence, tags = ss.text, ss.label
        sentence = sentence.cuda()
        tags = tags.cuda()
        
        output = model(sentence).permute(0,2,1)
        
        optimizer.zero_grad()
        loss = criterion(output, tags)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
    print(f'epoch {epoch+1}: {running_loss/len(train)}')

  3%|██▌                                                                               | 8/252 [00:00<00:03, 76.19it/s]

epoch 1: 0.04564186577860973


  3%|██▌                                                                               | 8/252 [00:00<00:03, 74.74it/s]

epoch 2: 0.03389671413763872


  3%|██▌                                                                               | 8/252 [00:00<00:03, 76.27it/s]

...

epoch 8: 0.023290865462355814


  3%|██▎                                                                               | 7/252 [00:00<00:03, 69.31it/s]

epoch 9: 0.02263103076783735


100%|████████████████████████████████████████████████████████████████████████████████| 252/252 [00:02<00:00, 85.28it/s]

epoch 10: 0.022074661168759214


In [24]:
with torch.no_grad():
    model.eval()
    acc = 0.0
    for ss in tqdm(trainloader):
        sentence, tags = ss.text, ss.label
        sentence = sentence.cuda()
        tags = tags.cuda()
        
        output = model(sentence).argmax(dim=-1)
        acc += torch.sum(output == tags)/tags.size(1)
    print(acc/len(train))

100%|███████████████████████████████████████████████████████████████████████████████| 252/252 [00:01<00:00, 229.49it/s]

tensor(0.8013, device='cuda:0')
